In [ ]:
from astroquery.mast import Observations  #Used to download IMA files from the MAST HST archive
from astropy.io import fits #Used to open the files
import matplotlib.pyplot as plt #Used to plot the images
import numpy as np #Used for some math

%matplotlib notebook 

### Get the table of observations associated to GO-14114 (PI van Dokkum, the first proposal to use the DASH mode)

In [ ]:
obsTable = Observations.query_criteria(proposal_id=['14114'],dataproduct_type="image",project='HST')

### Get the full list of products associated to the table and restric the list to IMA files

In [ ]:
product_list = Observations.get_product_list(obsTable)
BM = (product_list['productSubGroupDescription']  == 'IMA') 

product_list = product_list[BM]


### Display (part of) the IMA files list

In [ ]:
product_list.show_in_notebook(display_length=5)

### Pick a single IMA file to work on, download it and display it

In [ ]:
myID = product_list['obsID'][0:1]
download = Observations.download_products(myID,mrp_only=False,productSubGroupDescription=['IMA','DRZ'])

In [ ]:
localpathtofile = download['Local Path'][0]

In [ ]:
original_ima = fits.open(localpathtofile)
original_ima.info()

In [ ]:
nsamp = original_ima[0].header['NSAMP']
print('NSAMP',nsamp)
fig,axarr = plt.subplots(nsamp//4,4, figsize=(14,10))

for i in range(nsamp,0,-1):
    row = (i-1)//4
    col = (i-1)%4
    print(row,col,i)
    immed = np.nanmedian(original_ima['SCI',i].data)
    stdev = np.nanstd(original_ima['SCI',i].data)
    axarr[row,col].imshow(original_ima['SCI',i].data,clim=[immed-.3*stdev,immed+1.*stdev],cmap='Greys')
    axarr[row,col].set_title(str(i))
    axarr[row,col].set_xticks([]) 
    axarr[row,col].set_yticks([]) 
    
fig.tight_layout()